In [28]:
import os, re, laspy
import numpy as np
import pandas as pd
from datetime import datetime
import io

folder = "../raw_data/kijkduin_hourly/pointclouds"
files = sorted([f for f in os.listdir(folder) if f.endswith(".laz")])

def parse_timestamp(filename):
    # e.g. kijkduin_170117_120041.laz → 2017-01-17 12:00:41
    date_str = re.search(r"_(\d{6})_(\d{6})", filename)
    d, t = date_str.groups()
    return datetime.strptime(d + t, "%y%m%d%H%M%S")

data_list = []

for f in files:
    timestamp = parse_timestamp(f)
    path = os.path.join(folder, f)
    las = laspy.read(path)
    xyz = np.vstack((las.x, las.y, las.z)).T
    df = pd.DataFrame(xyz, columns=["x", "y", "z"])
    df["timestamp"] = timestamp
    data_list.append(df)

# Combine into one DataFrame (the 4D dataset)
pc_4d = pd.concat(data_list, ignore_index=True)


In [29]:
pc_4d

,x,y,z,timestamp
0,-240.41250,-29.42850,-37.80900,2017-01-17 12:00:41
1,-240.27375,-28.77525,-37.80500,2017-01-17 12:00:41
2,-240.05400,-27.03525,-37.78725,2017-01-17 12:00:41
3,-240.79400,-26.70250,-37.79650,2017-01-17 12:00:41
4,-239.69200,-26.80325,-37.78025,2017-01-17 12:00:41
...,...,...,...,...
10993045,-34.29225,-298.51075,-24.37775,2017-01-19 14:01:21
10993046,-34.20400,-299.78475,-24.21775,2017-01-19 14:01:21
10993047,-34.29225,-298.51075,-24.37775,2017-01-19 14:01:21
10993048,-34.14925,-299.30275,-24.44075,2017-01-19 14:01:21


In [ ]:
import open3d as o3d

# Convert first scan to reference
ref = o3d.geometry.PointCloud()
ref.points = o3d.utility.Vector3dVector(data_list[0][["x","y","z"]].values)

registered = []
for df in data_list[1:]:
    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(df[["x","y","z"]].values)
    reg = o3d.pipelines.registration.registration_icp(
        pc, ref, 0.2, np.eye(4),
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    pc.transform(reg.transformation)
    registered.append(np.asarray(pc.points))


In [3]:
from scipy.spatial import cKDTree

def elevation_change(pc_t1, pc_t2, grid_res=1.0):
    # Grid-based DoD (Difference of DEMs)
    # Create mesh grid
    x_min, x_max = pc_t1[:,0].min(), pc_t1[:,0].max()
    y_min, y_max = pc_t1[:,1].min(), pc_t1[:,1].max()
    xi = np.arange(x_min, x_max, grid_res)
    yi = np.arange(y_min, y_max, grid_res)
    xx, yy = np.meshgrid(xi, yi)
    # Compute mean z per grid for each time
    def mean_grid(pc):
        df = pd.DataFrame(pc, columns=["x","y","z"])
        grouped = df.groupby([df.x.round(), df.y.round()])["z"].mean()
        return grouped
    dem1 = mean_grid(pc_t1)
    dem2 = mean_grid(pc_t2)
    dod = dem1 - dem2
    return dod


(np.int64(0), np.int64(0))

In [23]:
path = "../raw_data/kijkduin_hourly/supplementary/KNMI_201611_201705_hourly_330_hoekvanholland_formatted.csv"
df = pd.read_csv(path)
df.YYYYMMDD.isna().sum(), df.HH.isna().sum()

# Now sort by the new datetime column
df_sorted = df.sort_values(['YYYYMMDD', 'HH']).reset_index(drop=True)
df_sorted

,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10,TD,...,VV,N,U,WW,IX,M,R,S,O,Y
0,330,20161001,1,180,60,70,80,140,NaN,114,...,NaN,NaN,84,NaN,6,NaN,NaN,NaN,NaN,NaN
1,330,20161001,2,190,50,50,70,129,NaN,112,...,NaN,NaN,89,NaN,6,NaN,NaN,NaN,NaN,NaN
2,330,20161001,3,160,50,50,60,126,NaN,112,...,NaN,NaN,91,NaN,6,NaN,NaN,NaN,NaN,NaN
3,330,20161001,4,170,50,50,70,120,NaN,111,...,NaN,NaN,94,NaN,6,NaN,NaN,NaN,NaN,NaN
4,330,20161001,5,170,60,60,80,121,NaN,111,...,NaN,NaN,94,NaN,6,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5803,330,20170530,20,240,80,80,120,157,NaN,121,...,NaN,NaN,79,NaN,6,NaN,NaN,NaN,NaN,NaN
5804,330,20170530,21,240,80,90,140,161,NaN,123,...,NaN,NaN,78,NaN,6,NaN,NaN,NaN,NaN,NaN
5805,330,20170530,22,240,80,80,130,161,NaN,127,...,NaN,NaN,80,NaN,6,NaN,NaN,NaN,NaN,NaN
5806,330,20170530,23,240,70,80,110,158,NaN,129,...,NaN,NaN,82,NaN,6,NaN,NaN,NaN,NaN,NaN


In [26]:
df_sorted.head(5).T

,0,1,2,3,4
STN,330.0,330.0,330.0,330.0,330.0
YYYYMMDD,20161001.0,20161001.0,20161001.0,20161001.0,20161001.0
HH,1.0,2.0,3.0,4.0,5.0
DD,180.0,190.0,160.0,170.0,170.0
FH,60.0,50.0,50.0,50.0,60.0
FF,70.0,50.0,50.0,50.0,60.0
FX,80.0,70.0,60.0,70.0,80.0
T,140.0,129.0,126.0,120.0,121.0
T10,NaN,NaN,NaN,NaN,NaN
TD,114.0,112.0,112.0,111.0,111.0
